# Load the dataset

Import necessary packages

In [2]:
#import packages
import torch
import torch.nn as nn
import numpy as np #array operations
import matplotlib.pyplot as plt #show image to check correct loading
import torch.nn.functional as F
import os #iterate throug directories, join paths
import sys
from torch import optim
import random
import pickle #store training set
import cv2

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

Load the dataset

In [3]:
#Unzipping

!unzip ppke-itk-neural-networks-otp-2020-challenge.zip

Streaming output truncated to the last 5000 lines.
  inflating: NN_2020_Kaggle_dataset/Algae.Chlorophyta.Schroederia/DHM202015100700020160204T1447170002.png  
  inflating: NN_2020_Kaggle_dataset/Algae.Chlorophyta.Schroederia/DHM202015100700020160204T1450350001.png  
  inflating: NN_2020_Kaggle_dataset/Algae.Chlorophyta.Schroederia/DHM202015100700020160204T1454010001.png  
  inflating: NN_2020_Kaggle_dataset/Algae.Chlorophyta.Schroederia/DHM202015100700020160208T0958140001.png  
  inflating: NN_2020_Kaggle_dataset/Algae.Chlorophyta.Schroederia/DHM202015100700020160208T1006000001.png  
  inflating: NN_2020_Kaggle_dataset/Algae.Chlorophyta.Schroederia/DHM202015100700020160208T1110120001.png  
  inflating: NN_2020_Kaggle_dataset/Algae.Chlorophyta.Schroederia/DHM202015100700020160208T1227410002.png  
  inflating: NN_2020_Kaggle_dataset/Algae.Chlorophyta.Schroederia/DHM202015100700020160212T1446430001.png  
  inflating: NN_2020_Kaggle_dataset/Algae.Chlorophyta.Schroederia/DHM2020151007000201

In [4]:
DATADIR = "./NN_2020_Kaggle_dataset"

CATEGORIES = sorted(os.listdir(DATADIR)) #categories are the names of the directories
CATEGORIES = CATEGORIES[0:16] #last 3 things are the test folder and 2 other files which we don't need

IMG_SIZE = 80
NUM_CLASSES = 16

In [12]:
#Build the dataset
dataset = []

def create_dataset():
    for category in CATEGORIES:

        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)  # get the classes  (0 to 15)
        print("Category name: %s, Category index: %d" % (category, class_num)) #check if names and indices are correct

        for img in os.listdir(path):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                dataset.append([new_array, class_num])  # add this to our dataset
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_dataset()

Category name: Algae.Bacillariophyta.Asterionella, Category index: 0
Category name: Algae.Bacillariophyta.Aulacoseira, Category index: 1
Category name: Algae.Bacillariophyta.Centrales, Category index: 2
Category name: Algae.Bacillariophyta.Nitzschia_acicularis, Category index: 3
Category name: Algae.Bacillariophyta.Nitzschia_palea, Category index: 4
Category name: Algae.Chlorophyta.Coelastrum, Category index: 5
Category name: Algae.Chlorophyta.Monoraphidium, Category index: 6
Category name: Algae.Chlorophyta.Pediastrum, Category index: 7
Category name: Algae.Chlorophyta.Scenedesmus, Category index: 8
Category name: Algae.Chlorophyta.Schroederia, Category index: 9
Category name: Algae.Chlorophyta.Tetraedron, Category index: 10
Category name: Algae.Chlorophyta.Tetrastrum, Category index: 11
Category name: Bacteria.Cyanobacteria.Anabaena_solitaria, Category index: 12
Category name: Bacteria.Cyanobacteria.Cyanobacteria_coenobia, Category index: 13
Category name: Bacteria.Cyanobacteria.Fila

Divide the data into training and validation dataset

In [13]:
# Import libraries
from keras.utils import to_categorical # Convert class vectors to binary class matrices.
from sklearn.model_selection import train_test_split

#ezt innentől tedd egy függvénybe majd, mert így csúúúúúúúúúúúúúnya
#def split_dataset():

X = []
y = []

# X forms the images, and y forms the labels
for features,label in dataset:
    X.append(features)
    y.append(label)

X=np.array(X)
y=np.array(y)

# Here I split original training data to sub-training (80%) and validation data (20%)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)

y_train = tf.keras.utils.to_categorical(y_train, num_classes=NUM_CLASSES)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=NUM_CLASSES)

In [14]:
# Prepare the training images
X_train = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X_train = X_train.astype('float32')
X_train /= 255

# Prepare the validation images
X_val = X_val.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X_val = X_val.astype('float32')
X_val /= 255

Prepare the test images

In [15]:
test_data = []
IMG_SIZE = 80

def create_test_data():
  path = os.path.join(DATADIR,"test")
  for img in os.listdir(path):  # iterate over each image
    try:
        img_array = cv2.imread(os.path.join(path,img))  # convert to array
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
        test_data.append(new_array)  # add this to our test_data
    except Exception as e:  # in the interest in keeping the output clean...
        pass

create_test_data()

# X_test forms the test images, and y_test forms the test labels
#X_test = np.array(test_data)
#y_test = to_categorical(test_data)

X_test = []

X_test=np.array(test_data)

# Prepare the test images
X_test = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X_test = X_test.astype('float32')
X_test /= 255

# Using pretrained models for image classification with feature extraction

**Simple model: ResNet50**

In [17]:
from tensorflow.keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

base_model = ResNet50(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights="imagenet") #néha ez nem működik, mert a colab úgy gondolja, hogy nem szeretné betölteni:)))

base_model.summary()



94773248/94765736 [==============================] - 0s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 86, 86, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 40, 40, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 40, 40, 64)   256         conv1_conv[0][0]                 
__________________________

In [18]:
# Preprocessing the input 
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [19]:
# Extracting features
train_features = base_model.predict(np.array(X_train), batch_size=32, verbose=1)
test_features = base_model.predict(np.array(X_test), batch_size=32, verbose=1)
val_features = base_model.predict(np.array(X_val), batch_size=32, verbose=1)

53/53 [==============================] - 34s 636ms/step


In [39]:
# Saving the features so that they can be used for future
np.savez("train_features", train_features, y_train)
np.savez("test_features", test_features, y_test)
np.savez("val_features", val_features, y_val)

In [21]:
# Current shape of features
print(train_features.shape, "\n",  test_features.shape, "\n", val_features.shape)

(6704, 3, 3, 2048) 
 (938, 3, 3, 2048) 
 (1676, 3, 3, 2048)


In [38]:
# Flatten extracted features based on the previously printed shape
train_features = np.reshape(train_features, (6704, 3*3*2048) )
test_features = np.reshape(test_features, (938, 3*3*2048))
val_features = np.reshape(val_features, (1676, 3*3*2048))

In [27]:
from keras.layers import Dense, Dropout
from keras.models import Model
from keras import models
from keras import layers
from keras import optimizers

# Add Dense and Dropout layers on top of ResNet50 pre-trained
model = models.Sequential()
model.add(layers.Dense(2048, activation='relu', input_dim=3*3*2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation="softmax"))

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 2048)              37750784  
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                32784     
Total params: 37,783,568
Trainable params: 37,783,568
Non-trainable params: 0
_________________________________________________________________


In [53]:
# Train the the model with batch size of 32 and 10 epochs on both training and validation data
history = model.fit(train_features, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          validation_data=(val_features, y_val))

Epoch 1/10
210/210 [==============================] - 46s 218ms/step - loss: 2.1532 - accuracy: 0.2925 - val_loss: 1.9706 - val_accuracy: 0.3150
Epoch 2/10
210/210 [==============================] - 46s 218ms/step - loss: 2.1402 - accuracy: 0.2942 - val_loss: 1.9079 - val_accuracy: 0.3204
Epoch 3/10
210/210 [==============================] - 46s 220ms/step - loss: 2.1465 - accuracy: 0.2958 - val_loss: 1.9381 - val_accuracy: 0.3174
Epoch 4/10
210/210 [==============================] - 46s 218ms/step - loss: 2.1462 - accuracy: 0.2976 - val_loss: 1.9983 - val_accuracy: 0.3156
Epoch 5/10
210/210 [==============================] - 46s 218ms/step - loss: 2.1525 - accuracy: 0.2964 - val_loss: 2.0097 - val_accuracy: 0.3162
Epoch 6/10
210/210 [==============================] - 46s 218ms/step - loss: 2.1524 - accuracy: 0.2947 - val_loss: 1.9650 - val_accuracy: 0.3174
Epoch 7/10
210/210 [==============================] - 46s 217ms/step - loss: 2.1379 - accuracy: 0.3028 - val_loss: 1.8821 - val_ac

**High-end model: InceptionResNetV2**


In [ ]:
#TODO

# Balancing and augmenting the dataset

Adaptive synthetic sampling approach for imbalanced learning (**ADASYN**):
ADASYN generates synthetic data for classes with less samples in a way that datasets that are more difficult to learn are generated more compared to samples that are easier to learn.

                                                              vaaaaaaaaaaaaaaaagy


Synthetic Minority Over-sampling Technique (**SMOTE**):
SMOTE involves over sampling the minority class and under sampling of the majority class to get the best results.

In [ ]:
#TODO

Data augmentation with:
*   Scaling
*   Cropping
*   Flipping
*   Rotation
*   Translation
*   Adding Noise
*   Changing lighting conditions

In [ ]:
#TODO

# Testing

In [54]:
def prepare(filepath):
    IMG_SIZE = 80 
    img_array = cv2.imread(filepath)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # return the image with shaping that TF wants.

path = "NN_2020_Kaggle_dataset/test"

prediction = model.predict(test_features)
predicted_classes = np.argmax(prediction, axis = 1)
with open('first_results.txt', 'w') as new_results:
  new_results.write("Id,Category\n")
  i = 0
  for img in os.listdir(path):  
    new_results.write("%s,%s\n" % (img, int(predicted_classes[i]))) #i. image, index of max value
    i = i+1

Note(s) to self:

*   rossz sorrendben van valszeg a prediction és a képek a for ciklusban, tehát ennek nincs értelme, lol
*   durva overfitting 8-ra és 12-re, mert unbalanced
*   csináld meg a TODO-kat, amit akarsz és kedvesebb lesz
*   Training és Validation set váltogatása a nagy datasetből ("eltolósan")
*   Learning Rate-tel játszadozni (meg egyéb paraméterekkel is)
*   Tensor Board?

